If this notebook is being run on Google Colab, install the required packages before anything else.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install plasmapy==0.7.0

# PlasmaPy Thomson Scattering Tutorial

In [ ]:
%matplotlib inline

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.diagnostics import thomson

# This function makes a plot of the Thomson scattering spectrum
def plot_result(wavelengths, alpha, Skw, iaw_range=np.array([528,536])*u.nm):
    # Estimate the max of the EPW signal for plotting
    a = np.argmin(np.abs(iaw_range[0]-wavelengths))
    b = np.argmin(np.abs(iaw_range[1]-wavelengths))
    iaw_max = np.max(Skw.value)
    epw_max = max([np.max(Skw[0:a].value), np.max(Skw[b:].value)])
    
    fig, ax = plt.subplots(ncols=2, figsize=(12, 6))
    fig.subplots_adjust(wspace=0.2)
    
    if alpha > 1:
        fig.suptitle(f"alpha = {alpha:.1f} (collective scattering)")
    else:
        fig.suptitle(f"alpha = {alpha:.1f} (noncollective scattering)")

    for x in ax:
        x.set_xlabel("Wavelength (nm)")
        x.set_ylabel("Skw")
        x.axvline(x=probe_wavelength.value, color="red", label="Probe wavelength")
    ax[0].tick_params(axis='both', which='major', labelsize=12)
    ax[0].set_xlim(520, 545)
    ax[0].set_ylim(0, 1.2*epw_max)
    ax[0].set_title("Electron Plasma Wave", fontsize=14)
    ax[0].plot(wavelengths.value, Skw)
    ax[0].legend(loc='upper left', fontsize=12)

    ax[1].tick_params(axis='both', which='major', labelsize=12)
    ax[1].set_xlim(530, 534)
    ax[1].set_ylim(0, 1.2*iaw_max)
    ax[1].set_title("Ion Acoustic Wave", fontsize=14)
    ax[1].plot(wavelengths.value, Skw)
    ax[1].legend(loc='upper left', fontsize=12);



[PlasmaPy documentation]:https://docs.plasmapy.org/en/latest/ad/diagnostics/thomson.html
[spectral_density]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.thomson.spectral_density.html#plasmapy.diagnostics.thomson.spectral_density

This notebook provides a simple example using the [spectral_density] function in PlasmaPy to model Thomson scattering. More examples and documentation can be found in the [PlasmaPy documentation]


The first step is to define the scattering geometry, the wavelength of the probe laser, and the wavelength range of the detector

In [ ]:
# The probe wavelength can in theory be anything, but in practice integer frequency multiples of the Nd:YAG wavelength
# 1064 nm are used (532 corresponds to a frequency-doubled probe beam from such a laser).
probe_wavelength = 532 * u.nm

# Array of wavelengths over which to calculate the spectral distribution

wavelengths = (
    np.linspace(probe_wavelength.value - 30, probe_wavelength.value + 30, num=5000)
    * u.nm
)


# The scattering geometry is defined by unit vectors for the orientation of the probe laser beam (probe_n) and
# the path from the scattering volume (where the measurement is made) to the detector (scatter_n).
# These can be setup for any experimental geometry - this geometry is typical on the OMEGA laser facility.
probe_vec = np.array([1, 0, 0])
scattering_angle = np.deg2rad(63)
scatter_vec = np.array([np.cos(scattering_angle), np.sin(scattering_angle), 0])

Now we will specify a set of plasma parameters, call the `spectral_density` function, and plot the results

In [ ]:
n = 2e17 * u.cm ** -3
Te = 12 * u.eV
Ti = 10 * u.eV
ion_species = 'H+'

alpha, Skw = thomson.spectral_density(
    wavelengths,
    probe_wavelength,
    n,
    Te,
    Ti,
    ion_species = ion_species,
    probe_vec=probe_vec,
    scatter_vec=scatter_vec,
)

plot_result(wavelengths, alpha, Skw)

The spectral density function can also include multiple populations of ions and electrons, each of which can have a separate drift velocity in three dimensions. Here's a more complex example.

In [ ]:
n = 2e17 * u.cm ** -3

efract = np.array([0.5, 0.5])
Te = np.array([20, 6]) * u.eV
electron_vel = np.array([[0,0,0], [1000, 0, 0]])*u.km/u.s


ifract = np.array([0.2, 0.8])
ion_species = ["He-4 1+", "C-12 5+"]
Ti = np.array([10, 15]) * u.eV
ion_vel = np.array([[300,0,0], [-900, 0, 0]])*u.km/u.s

alpha, Skw = thomson.spectral_density(
    wavelengths,
    probe_wavelength,
    n,
    Te,
    Ti,
    efract = efract,
    ifract = ifract, 
    ion_species = ion_species,
    electron_vel=electron_vel,
    ion_vel=ion_vel,
    probe_vec=probe_vec,
    scatter_vec=scatter_vec,
)

plot_result(wavelengths, alpha, Skw)